In [13]:
import janitor
import pandas as pd

## 2.1 利用also()方法穿插执行任意函数

In [15]:
df = (
    # 构造示例数据框
    pd.DataFrame({"a": [1, 2, 3], "b": list("abc")})
    .query("a > 1")
    # 利用also()插入lambda函数接受上一步的输入对象
    .also(lambda df: print(f"a字段<=1的记录有{df.query('a <= 1').shape[0]}行"))
    .rename(columns={'a': 'new_a'})
    # 利用also()实现中间计算结果的导出
    .also(lambda df: df.to_csv("temp.csv", index=False))
    # 利用also()打印到这一步时数据框计算结果的字段名
    .also(
        lambda df: print(f"字段名：{df.columns.tolist()}")
    )
    .drop(columns='b')
)
df

a字段<=1的记录有0行
字段名：['new_a', 'b']


,new_a
1,2
2,3


## 2.2 利用case_when()方法实现多条件分支

In [27]:
df = pd.DataFrame(
    {
        "a": [0, 0, 1, 2],
        "b": [0, 3, 4, 5],
        "c": [6, 7, 8, 9],
    }
)
df

,a,b,c
0,0,0,6
1,0,3,7
2,1,4,8
3,2,5,9


In [29]:
df.case_when(
    ((df.a == 0) & (df.b == 0)), '类别1',
    ((df.a == 0) & (df.b != 0)), '类别2',
    # 其他情况
    '类别3',
    column_name="类别",
)

,a,b,c,类别
0,0,0,6,类别1
1,0,3,7,类别2
2,1,4,8,类别3
3,2,5,9,类别3


## 2.3 利用conditional_join()实现条件连接

In [34]:
# 定义示例左表
df_left = pd.DataFrame({
    'id': list('abcd'),
    'left_range_start': [2, 9, 14, 30],
    'left_range_end': [5, 11, 21, 35]
})
df_left

,id,left_range_start,left_range_end
0,a,2,5
1,b,9,11
2,c,14,21
3,d,30,35


In [35]:
# 定义示例右表
df_right = pd.DataFrame({
    'id': list('ijxy'),
    'right_range_start': [2, 6, 15, 28],
    'right_range_end': [3, 10, 18, 31]
})
df_right

,id,right_range_start,right_range_end
0,i,2,3
1,j,6,10
2,x,15,18
3,y,28,31


In [41]:
(
    df_left
    .conditional_join(
        df_right,
        # 满足left_range_start <= right_range_start
        ('left_range_start', 'right_range_start', '<='),
        # 且满足left_range_end >= right_range_end
        ('left_range_end', 'right_range_end', '>=')
    )
)

left                                 right                                  
    id left_range_start left_range_end    id right_range_start right_range_end
0    a                2              5     i                 2               3
1    c               14             21     x                15              18

## 2.4 利用move()方法快捷完成字段位置调整

In [60]:
df = pd.DataFrame({"a": [2, 4, 6], "b": [1, 3, 5], "c": [7, 8, 9]})
df

,a,b,c
0,2,1,7
1,4,3,8
2,6,5,9


In [61]:
# 将a字段挪动到c字段之前
df.move(source="a", target="c", position="before", axis=1)

,b,a,c
0,1,2,7
1,3,4,8
2,5,6,9


In [62]:
# 将a字段挪动到c字段之后
df.move(source="a", target="c", position="after", axis=1)

,b,c,a
0,1,7,2
1,3,8,4
2,5,9,6
